In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_path = "/content/drive/MyDrive"
img_path = drive_path + "/atml_bucket"


In [ ]:
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import ast
import os
import shutil
import sys

In [ ]:
df = pd.read_csv(drive_path+"/ATML_Dataset_Largest.csv")
df['bbox'] = df.bbox.apply(lambda x: ast.literal_eval(str(x)))
df['Class'] = df.Class.apply(lambda x: ast.literal_eval(str(x)))
df_train, df_val = train_test_split(df, test_size=0.33, random_state=42, shuffle=True)

In [ ]:
print(df_train)

In [ ]:
labels_mapping = {
    'airport': 0,
    'power_plant': 1,
    'stadium': 2
}

In [ ]:
def segregate_data(df, new_img_path, label_path):
  for i in range(len(df["fileName"])):
    try:
      # if(os.path.exists(new_img_path+df.iloc[i]["fileName"]+'.tif') and os.path.exists(label_path+df.iloc[i]["fileName"]+'.txt')):
      #   continue
      yolo_list = []
      im = Image.open(img_path + "/"+ df.iloc[i]["fileName"].lower() + '.tif')
      im_array = np.array(im).shape
      img_width = im_array[0]
      img_height = im_array[1]
      for j in range(len(df.iloc[i]["bbox"])):
        boundary = df.iloc[i]["bbox"][j]
        x_min = boundary[0]
        y_min = boundary[1]
        x_max = boundary[2]
        y_max = boundary[3]
        x_center_norm = abs(x_max + x_min) / (2*img_width)    
        y_center_norm = abs(y_max + y_min) / (2*img_height)
        width_norm = abs(x_max - x_min) / img_width
        height_norm = abs(y_max - y_min) / img_height
        yolo_list.append([labels_mapping[df.iloc[i]["Class"][j]], x_center_norm, y_center_norm, width_norm, height_norm])

  
      txt_filename = os.path.join(label_path, str(df.iloc[i]["fileName"]) + ".txt")
      np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])
      #shutil.copyfile(os.path.join(img_path,df.iloc[i]["fileName"].lower()+'.tif'), os.path.join(new_img_path,df.iloc[i]["fileName"]+'.tif'))
    except FileNotFoundError as e:
      print("File "+df.iloc[i]["fileName"]+" not found!")
      continue

In [ ]:
segregate_data(df_train,drive_path+"/atml_project_data/train/images",drive_path+"/atml_project_data/train/labels")

In [ ]:
segregate_data(df_val,drive_path+"/atml_project_data/test/images",drive_path+"/atml_project_data/test/labels")
print(df_train.shape)
print(df_val.shape)
print(len(os.listdir("/content/drive/MyDrive/atml_project_data/train/images")))
print(len(os.listdir("/content/drive/MyDrive/atml_project_data/train/labels")))
print(len(os.listdir("/content/drive/MyDrive/atml_project_data/test/images")))
print(len(os.listdir("/content/drive/MyDrive/atml_project_data/test/labels")))

In [ ]:
!git clone  'https://github.com/ultralytics/yolov5.git'
!pip install -qr '/content/yolov5/requirements.txt'  # install dependencies
## Create a yaml file and move it into the yolov5 folder ##
model_path = drive_path+'/atml_project_data/model'
shutil.copyfile(model_path+'/bcc.yaml', '/content/yolov5/bcc.yaml')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/
!python yolov5/train.py --img 640 --batch 8 --epochs 100 \
--data /content/yolov5/bcc.yaml --cfg /content/yolov5/models/yolov5s.yaml --name BCCM --cache --freeze 10

In [ ]:
## TO PREDICT IMAGES IN A FOLDER ##
best_weight = '' # you have to input the best weight part here.
!python yolov5/detect.py --source /content/yolov5/runs/BCCM
--weights /content/yolov5/runs/BCCM/best.pt
--output /content/inference/output